In [1]:
import tensorflow as tf
import numpy as np

In [2]:
a = np.random.randint(1,20,(3,4,2))
a

array([[[19,  9],
        [ 1,  8],
        [16, 19],
        [14, 14]],

       [[11, 11],
        [11, 10],
        [16, 17],
        [17, 12]],

       [[18,  7],
        [11, 17],
        [11, 14],
        [15, 18]]])

array 'a' has a batch size of 3, sequence of length 4(4 time steps) and each seqeunce of length 2

In [3]:
tf.reset_default_graph()

tf_input_a  = tf.constant(a, dtype = tf.float32)
lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units = 35)
outputs,state = tf.nn.dynamic_rnn(cell=lstm_cell, dtype = tf.float32, inputs = tf_input_a)

print(outputs)
print(state)

Tensor("rnn/transpose_1:0", shape=(3, 4, 35), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(3, 35) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(3, 35) dtype=float32>)


Shape of the output is batch_szie x sequence_length x num_units. state.h is the output of the last cell and state.c is the cells state of output cell. Both have dimensions: batch_size x num_units

In [4]:
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    output_val,state_val=sess.run([outputs,state])

Further, output of the last cell is equal to the state.h. which can be verifed as:

In [5]:
output_val[:,-1] == state_val.h

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True]])

## Bidirectional RNNs

In [6]:
tf.reset_default_graph()

tf_input_b  = tf.constant(a, dtype = tf.float32)
forward_cell = tf.nn.rnn_cell.LSTMCell(num_units = 35)
backward_cell = tf.nn.rnn_cell.LSTMCell(num_units = 40) 
# num_units are created different in forward and backward cell just to show the effect on shape
(output_fw,output_bw),(state_fw,state_bw) = tf.nn.bidirectional_dynamic_rnn(cell_fw=forward_cell,
                                                                           cell_bw = backward_cell,
                                                                           inputs=tf_input_b,
                                                                           dtype = tf.float32)

In [7]:
output_fw.shape,output_bw.shape

(TensorShape([Dimension(3), Dimension(4), Dimension(35)]),
 TensorShape([Dimension(3), Dimension(4), Dimension(40)]))

In [8]:
state_fw.h.shape,state_bw.h.shape

(TensorShape([Dimension(3), Dimension(35)]),
 TensorShape([Dimension(3), Dimension(40)]))

In [9]:
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    (output_fw_val,output_bw_val),(state_fw_val,state_bw_val)=sess.run([(output_fw,output_bw),(state_fw,state_bw)])

In [10]:
output_fw_val[:,-1] == state_fw_val.h 

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True]])

It seems bidirectional RNN behaves similar to the normal RNN.

In [11]:
output_bw_val[:,-1] == state_bw_val.h 

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False]])

Output of the backward cell doesn't seem to follow the trend. Since the flow is backward, tensorflow has reversed the indices. 
Last cell has index 0 now. 

In [12]:
output_bw_val[:,0] == state_bw_val.h 

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True]])